# ResNet CIFAR-10 图像分类实战

本笔记本展示如何使用 **PyTorch** 和 **ResNet-18** 模型对 **CIFAR-10** 数据集进行图像分类。
最后我们将使用 **Seaborn** 绘制训练过程中的损失（Loss）和准确率（Accuracy）曲线。

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time

# 配置 Seaborn 绘图风格
sns.set_theme(style="whitegrid", palette="muted")

# 自动选择设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"当前使用设备: {device}")

## 1. 数据准备与预处理

这里我们定义数据增强策略（随机裁剪、水平翻转）以提高模型泛化能力，并进行标准化处理。
数据集将自动下载到 `./data` 目录。

In [ ]:
# 超参数配置
BATCH_SIZE = 128
EPOCHS = 15
LEARNING_RATE = 0.01

# 训练集：增强 + 归一化
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 测试集：仅归一化
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# 加载数据
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## 2. 构建 ResNet 模型

使用 `torchvision.models` 提供的 `resnet18`。由于 CIFAR-10 只有 10 个类别，我们需要修改全连接层（FC Layer）。
我们使用预训练参数（`pretrained=True`）来加速收敛。

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

# 修改输出层为 10 类
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model = model.to(device)

## 3. 优化器与损失函数

使用交叉熵损失 (`CrossEntropyLoss`) 和 SGD 优化器。配合学习率调度器 (`StepLR`) 在训练后期降低学习率。

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

## 4. 训练与评估循环

定义训练和测试函数，记录每个 Epoch 的 Loss 和 Accuracy。

In [ ]:
train_losses, test_losses = [], []
train_accs, test_accs = [], []

def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(trainloader)
    epoch_acc = 100. * correct / total
    train_losses.append(epoch_loss)
    train_accs.append(epoch_acc)
    return epoch_loss, epoch_acc

def test(epoch):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
    epoch_loss = running_loss / len(testloader)
    epoch_acc = 100. * correct / total
    test_losses.append(epoch_loss)
    test_accs.append(epoch_acc)
    return epoch_loss, epoch_acc

### 开始训练

In [ ]:
print(f"开始训练 {EPOCHS} 个 Epochs...")
start_time = time.time()

for epoch in range(EPOCHS):
    t_loss, t_acc = train(epoch)
    v_loss, v_acc = test(epoch)
    scheduler.step()
    print(f"Epoch {epoch+1:02d} | Train Loss: {t_loss:.4f} Acc: {t_acc:.2f}% | Test Loss: {v_loss:.4f} Acc: {v_acc:.2f}%")

print(f"训练完成，总耗时: {(time.time() - start_time)/60:.2f} 分钟")

## 5. 可视化训练结果

利用 **Seaborn** 绘制 Loss 和 Accuracy 随 Epoch 变化的曲线，直观评估模型效果。

In [ ]:
epochs_range = range(1, EPOCHS + 1)

plt.figure(figsize=(14, 6))

# 绘制 Loss 曲线
plt.subplot(1, 2, 1)
sns.lineplot(x=epochs_range, y=train_losses, label='Train Loss', marker='o')
sns.lineplot(x=epochs_range, y=test_losses, label='Test Loss', marker='o')
plt.title('Training & Test Loss', fontsize=15)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# 绘制 Accuracy 曲线
plt.subplot(1, 2, 2)
sns.lineplot(x=epochs_range, y=train_accs, label='Train Acc', marker='o')
sns.lineplot(x=epochs_range, y=test_accs, label='Test Acc', marker='o')
plt.title('Training & Test Accuracy', fontsize=15)
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()